# 2 - Limpeza de dados

- **Missing values**: Não identificamos nenhum valor ausente no dataset;
- **Outliers**: Não identificamos nenhum outlier no dataset;
- **Inconsistência**: Pensamos em separar todos os dados relacionados à medidas de poluição, e indicadores de saúde em duas tabelas;
- **Padronização**: Para analisar os dados das medidas de poluição, precisamos separar por ano e estação, pois nossas hipóteses propostas utilizam a estação como parâmetro. O período de avaliação está bem variado, alguns indicadores possuem início em um ano, e terminam em outro. Outros indicadores são uma média do ano inteiro, e alguns foram avaliados mais vezes em locais diferentes.

In [ ]:
import pandas as pd
import os

input_path = "air_quality.parquet"
out_folder = "out"

os.makedirs(out_folder, exist_ok=True)
df = pd.read_parquet("air_quality.parquet")

Primeiro agrupamos usando a média, de todos os indicadores com mesmo nome e mesmo período. Para termos apenas um cálulo de determinado período. Isso acontece, pois o mesmo indicador foi avaliado em vários lugares da cidade. Mantemos apenas as colunas importantes no agrupamento.

In [ ]:

df = df.groupby(["Indicator ID", "Name", "Measure", "Measure Info", "Time Period"], as_index=False)["Data Value"].mean()

Agora precisamos separar todos os dados por estação do ano e ano. Ou seja, Time Period irá virar as colunas Season e Year \
Casos diferentes de Time Period e como tratamos: \
    - `Summer YYYY` : Apenas separamos em duas colunas. \
    - `Winter YYYY-YY`: Como o inverno começa em um ano e termina em outro, consideramos o ano em que ele começa, então mantemos o primeiro ano. \
    - `YYYY-YYYY`: Uma cópia para cada ano do intervalo (transforma no formato `YYYY`). \
    - `YYYY | Annual Average YYYY`: Uma cópia para summer do mesmo ano, e outra para o winter do ano passado.


In [ ]:
mask_season = (
    df['Time Period'].str.startswith('Winter') |
    df['Time Period'].str.startswith('Summer')
)

df_season = df[mask_season].copy()

# Summer YYYY é simples, apenas separar em duas colunas
df_season[['Season', 'Year']] = df_season['Time Period'].str.split(' ', expand=True)

# Winter YYYY-YY, manter apenas o primeiro ano
df_season['Year'] = df_season['Year'].str.split('-', expand=True)[0]
df_season.drop(columns=['Time Period'], inplace=True)
df_season['Year'] = df_season['Year'].astype(int)

# Pegando todos os intervalos de anos, para explodi-los
mask_year_range = df['Time Period'].str.contains(r'^\d{4}-\d{4}$', regex=True)
df_year_range = df[mask_year_range].copy()
df_year_range[['Start Year', 'End Year']] = df_year_range['Time Period'].str.split('-', expand=True).astype(int)
df_year_range['Year Count'] = df_year_range['End Year'] - df_year_range['Start Year'] + 1
df_year_range['Year List'] = df_year_range.apply(
    lambda row: list(range(row['Start Year'], row['End Year'] + 1)), axis=1
)

# Explode os intervalos dos anos
df_year_range = df_year_range.explode('Year List').copy()
df_year_range.rename(columns={'Year List': 'Year'}, inplace=True)
df_year_range = df_year_range[['Indicator ID', 'Name', 'Measure', 'Measure Info', 'Year', 'Data Value']]

# Pegando apenas os anos individuais
df_individual_years = df[~mask_season & ~mask_year_range].copy()
df_individual_years['Time Period'] = df_individual_years['Time Period'].str.replace('Annual Average ', '')
df_individual_years.rename(columns={'Time Period': 'Year'}, inplace=True)

# Juntando os anos individuais com os intervalos explodidos
df_year = pd.concat([df_individual_years, df_year_range], ignore_index=True)

# Separando ano em duas estações
df_year_summer = df_year.copy()
df_year_summer['Season'] = 'Summer'
df_year_summer['Year'] = df_year_summer['Year'].astype(int)

# O inverno é atribuido ao ano anterior
df_year_winter = df_year.copy()
df_year_winter['Season'] = 'Winter'
df_year_winter['Year'] = df_year_winter['Year'].astype(int) - 1

# Juntando todos os dados e fazendo a media de Data Value
df_season = pd.concat([df_season, df_year_summer, df_year_winter], ignore_index=True)
df_season = df_season.groupby(["Indicator ID", "Name", "Measure", "Measure Info", "Season", "Year"], as_index=False)["Data Value"].mean()
df_season.to_parquet(f"{out_folder}/season_air_quality.parquet", index=False)
del df_year, df_year_range, df_year_summer, df_year_winter, df_individual_years
df_season

,Indicator ID,Name,Measure,Measure Info,Season,Year,Data Value
0,365,Fine particles (PM 2.5),Mean,mcg/m3,Summer,2009,11.040496
1,365,Fine particles (PM 2.5),Mean,mcg/m3,Summer,2010,11.137837
2,365,Fine particles (PM 2.5),Mean,mcg/m3,Summer,2011,11.185284
3,365,Fine particles (PM 2.5),Mean,mcg/m3,Summer,2012,10.050035
4,365,Fine particles (PM 2.5),Mean,mcg/m3,Summer,2013,9.847340
...,...,...,...,...,...,...,...
392,661,Asthma hospitalizations due to Ozone,Estimated annual rate (age 18+),"per 100,000 adults",Winter,2014,4.070833
393,661,Asthma hospitalizations due to Ozone,Estimated annual rate (age 18+),"per 100,000 adults",Winter,2015,4.070833
394,661,Asthma hospitalizations due to Ozone,Estimated annual rate (age 18+),"per 100,000 adults",Winter,2016,3.400000
395,661,Asthma hospitalizations due to Ozone,Estimated annual rate (age 18+),"per 100,000 adults",Winter,2017,2.729167
